In [ ]:
import rerun as rr
from lerobot.cameras.opencv.configuration_opencv import OpenCVCameraConfig
from lerobot.teleoperators.so_leader import SO101LeaderConfig, SO101Leader
from lerobot.robots.so_follower import SO101FollowerConfig, SO101Follower

camera_config = {
    "front_1": OpenCVCameraConfig(index_or_path="/dev/follower_cam_1", width=640, height=480, fps=30, fourcc="MJPG"),
    "top_1": OpenCVCameraConfig(index_or_path="/dev/top_cam_1", width=640, height=480, fps=30, fourcc="MJPG"),
    "top_2": OpenCVCameraConfig(index_or_path="/dev/top_cam_2", width=640, height=480, fps=30, fourcc="MJPG"),
}

robot_config = SO101FollowerConfig(
    port="/dev/follower_arm_1",
    id="my_so101_follower_1",
    cameras=camera_config
)

teleop_config = SO101LeaderConfig(
    port="/dev/leader_arm_1",
    id="my_so101_leader_1",
)

robot = SO101Follower(robot_config)
teleop_device = SO101Leader(teleop_config)


In [2]:
rr.init("lerobot-teleop", spawn=True)
robot.connect()
teleop_device.connect()

[2026-02-09T10:45:45Z INFO  re_grpc_server] Listening for gRPC connections on 0.0.0.0:9876. Connect by running `rerun --connect rerun+http://127.0.0.1:9876/proxy`
[2026-02-09T10:45:45Z INFO  winit::platform_impl::linux::x11::window] Guessed window scale factor: 2
[2026-02-09T10:45:45Z WARN  wgpu_hal::vulkan::instance] Unable to find extension: VK_EXT_physical_device_drm
[2026-02-09T10:45:45Z WARN  wgpu_hal::gles::egl] No config found!
[2026-02-09T10:45:45Z WARN  wgpu_hal::gles::egl] EGL says it can present to the window but not natively
[2026-02-09T10:45:46Z WARN  wgpu_hal::gles::adapter] Max vertex attribute stride unknown. Assuming it is 2048
[2026-02-09T10:45:46Z WARN  wgpu_hal::gles::adapter] Max vertex attribute stride unknown. Assuming it is 2048
[2026-02-09T10:45:46Z INFO  egui_wgpu] There were 4 available wgpu adapters: {backend: Vulkan, device_type: DiscreteGpu, name: "NVIDIA GeForce RTX 5060 Laptop GPU", driver: "NVIDIA", driver_info: "580.126.09", vendor: NVIDIA (0x10DE), de

In [3]:
while True:
    observation = robot.get_observation()
    action = teleop_device.get_action()
    robot.send_action(action)
    
    # Log images to rerun
    for name in ["front_1", "top_1", "top_2"]:
        if name in observation:
            rr.log(f"camera/{name}", rr.Image(observation[name]))

[2026-02-09T10:49:53Z INFO  re_viewer::app] Reached memory limit of 17.1 GiB, dropping oldest data.
Traceback (most recent call last):
  File "/home/jinhyuk2me/miniforge3/envs/lerobot/bin/rerun", line 7, in <module>
    sys.exit(main())
  File "/home/jinhyuk2me/miniforge3/envs/lerobot/lib/python3.10/site-packages/rerun_sdk/rerun_cli/__main__.py", line 35, in main
    return subprocess.call([target_path, *sys.argv[1:]])
  File "/home/jinhyuk2me/miniforge3/envs/lerobot/lib/python3.10/subprocess.py", line 347, in call
    return p.wait(timeout=timeout)
  File "/home/jinhyuk2me/miniforge3/envs/lerobot/lib/python3.10/subprocess.py", line 1209, in wait
    return self._wait(timeout=timeout)
  File "/home/jinhyuk2me/miniforge3/envs/lerobot/lib/python3.10/subprocess.py", line 1959, in _wait
    (pid, sts) = self._try_wait(0)
  File "/home/jinhyuk2me/miniforge3/envs/lerobot/lib/python3.10/subprocess.py", line 1917, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt

KeyboardInterrupt: 

[2026-02-09T10:49:59Z ERROR re_grpc_client::write] Write messages call failed: gRPC error, message: "transport error"


In [4]:
robot.disconnect()
teleop_device.disconnect()